In [45]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPRegressor

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

from sklearn.model_selection import train_test_split

import nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
full_train = pd.read_json('/content/drive/MyDrive/1 MIDS/W207 Applied Machine Learning/Final Project/train.json', lines=True)
full_train = full_train.set_index(keys='index')
full_train.info()
full_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 0 to 2399
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2400 non-null   object 
 1   sequence             2400 non-null   object 
 2   structure            2400 non-null   object 
 3   predicted_loop_type  2400 non-null   object 
 4   signal_to_noise      2400 non-null   float64
 5   SN_filter            2400 non-null   int64  
 6   seq_length           2400 non-null   int64  
 7   seq_scored           2400 non-null   int64  
 8   reactivity_error     2400 non-null   object 
 9   deg_error_Mg_pH10    2400 non-null   object 
 10  deg_error_pH10       2400 non-null   object 
 11  deg_error_Mg_50C     2400 non-null   object 
 12  deg_error_50C        2400 non-null   object 
 13  reactivity           2400 non-null   object 
 14  deg_Mg_pH10          2400 non-null   object 
 15  deg_pH10             2400 non-null   o

,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
index,,,,,,,,,,,,,,,,,,
0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [8]:
full_test = pd.read_json('/content/drive/MyDrive/1 MIDS/W207 Applied Machine Learning/Final Project/test.json', lines=True)
full_test = full_test.set_index(keys='index')
full_test.info()
full_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3634 entries, 0 to 3633
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   3634 non-null   object
 1   sequence             3634 non-null   object
 2   structure            3634 non-null   object
 3   predicted_loop_type  3634 non-null   object
 4   seq_length           3634 non-null   int64 
 5   seq_scored           3634 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 198.7+ KB


,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
index,,,,,,
0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


The test set is a mixture of 'private test' and 'public test' data, differ by seq_length/ seq_score, should seperate the two sets. 

In [9]:
public_test = full_test[full_test.seq_length==107]
private_test = full_test[full_test.seq_length==130]
print(public_test.shape)
print(private_test.shape)


(629, 6)
(3005, 6)


In [6]:
perc =[.20, .40, .60, .80]
include =['object', 'float', 'int']
descriptive_summary = full_train.describe(percentiles = perc, include = include)

descriptive_summary

,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
count,2400,2400,2400,2400,2400.000000,2400.000000,2400.0,2400.0,2400,2400,2400,2400,2400,2400,2400,2400,2400,2400
unique,2400,2400,1507,1530,NaN,NaN,NaN,NaN,2394,2394,2389,2390,2387,2395,2394,2391,2392,2385
top,id_8c53bd1ba,GGAAAGUGAAAGGAAUCACGGAAACAAGAGAAGCUAUUUAGGACGU...,.................................................,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...,NaN,NaN,NaN,NaN,"[146151.225, 146151.225, 146151.225, 146151.22...","[104235.1742, 104235.1742, 104235.1742, 104235...","[314833.5695, 314833.5695, 314833.5695, 314833...","[171525.3217, 171525.3217, 171525.3217, 171525...","[271158.604, 271158.604, 271158.604, 271158.60...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
freq,1,1,278,278,NaN,NaN,NaN,NaN,6,6,10,9,13,6,6,10,9,15
mean,NaN,NaN,NaN,NaN,4.530456,0.662083,107.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,2.835142,0.473099,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,-0.103000,0.000000,107.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20%,NaN,NaN,NaN,NaN,1.856800,0.000000,107.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40%,NaN,NaN,NaN,NaN,3.715200,1.000000,107.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,4.442500,1.000000,107.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The data is structured quite complicated with different length for sequences can be different in different data set, so just double check that length is expected. 

In [15]:
def check_seq_length (data,seq, expected_length):
  return data.apply(lambda x:len(x[seq]) == x[expected_length], axis = 1)

for seq in ['sequence', 'structure', 'predicted_loop_type']: 
  print(f'Is the length for {seq} as expected?', all(check_seq_length(full_train, seq, 'seq_length')))


for seq in ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']: 
  print(f'Is the length for {seq} as expected?', all(check_seq_length(full_train, seq, 'seq_scored')))



Is the length for sequence as expected? True
Is the length for structure as expected? True
Is the length for predicted_loop_type as expected? True
Is the length for reactivity as expected? True
Is the length for deg_Mg_pH10 as expected? True
Is the length for deg_pH10 as expected? True
Is the length for deg_Mg_50C as expected? True
Is the length for deg_50C as expected? True


In [28]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def tokenise(data, dic):
  temp = list(data.sequence) + list(data.structure) +list(data.predicted_loop_type)
  temp = np.array(temp)

  k = np.array(list(dic.keys()))
  v = np.array(list(dic.values()))

  # Get argsort indices
  sidx = k.argsort()

  ks = k[sidx]
  vs = v[sidx]
  return vs[np.searchsorted(ks,temp)]

In [29]:
full_train['feature_array'] = full_train.apply(lambda x: tokenise(x, token2int), axis = 1)

In [43]:
np.array(full_train['feature_array'].tolist()).shape

(2400, 321)

In [47]:
 # split into a training and a dev data 
 
 train_data, dev_data = train_test_split(
    full_train, test_size=.1, random_state=34)
 train_data

,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,feature_array
index,,,,,,,,,,,,,,,,,,,
1449,id_9bdb90d72,GGAAAAGGGACCCAGGGCCCAGGAGACGACGGCCCAGGGAAACCGG...,......(((.(((.((((((.((.(.((.((((((.((....))))...,EEEEEESSSBSSSBSSSSSSBSSBSBSSBSSSSSSBSSHHHHSSSS...,1.468,1,107,68,"[0.5143, 0.6874, 0.451, 0.41140000000000004, 0...","[0.8756, 0.9918, 0.3546, 0.7176, 0.44060000000...","[0.9508000000000001, 1.117, 0.3112000000000000...","[0.5732, 0.8508, 0.5217, 0.6385000000000001, 0...","[0.9208000000000001, 1.0903, 0.54, 0.7169, 0.5...","[0.9189, 3.6807, 1.6154000000000002, 1.4268, 0...","[1.4156, 3.5641, 0.17650000000000002, 2.1167, ...","[2.1488, 5.6183, 0.1474, 1.149, 1.1569, 0.1784...","[0.4713, 3.4487, 1.1675, 2.2906, 0.74120000000...","[1.3601, 3.8044000000000002, 0.65, 1.601500000...","[5, 5, 3, 3, 3, 3, 5, 5, 5, 3, 4, 4, 4, 3, 5, ..."
393,id_29fb577aa,GGAAACCAAAUUGGAAUGGAAAUAACCUUUGGAAAAACCAUGCAAU...,.....((((...((...((......))..(((.....)))(((......,EEEEESSSSBBBSSMMMSSHHHHHHSSMMSSSHHHHHSSSSSSHHH...,7.817,1,107,68,"[0.11280000000000001, 0.12610000000000002, 0.1...","[0.20720000000000002, 0.32370000000000004, 0.1...","[0.23950000000000002, 0.2086, 0.14170000000000...","[0.1496, 0.2179, 0.1531, 0.1119, 0.0903, 0.080...","[0.1786, 0.2197, 0.1776, 0.1489, 0.1247, 0.103...","[0.28190000000000004, 0.633, 0.864100000000000...","[0.32330000000000003, 1.7866, 0.4002, 0.2553, ...","[2.4623, 2.1036, 0.9784, 0.5796, 0.708, 0.3378...","[0.5494, 2.2314, 1.1257, 0.5707, 0.3517, 0.276...","[0.5123, 1.5791, 1.1492, 0.8189000000000001, 0...","[5, 5, 3, 3, 3, 4, 4, 3, 3, 3, 6, 6, 5, 5, 3, ..."
2291,id_f444474b3,GGAAAACAACAAGGCCAAACUGUCACUAAGAAAUCUGCUGCUGAGG...,............(((......)))..............((((((((...,EEEEEEEEEEEESSSHHHHHHSSSMMMMMMMMMMMMMMSSSSSSSS...,4.855,1,107,68,"[0.1399, 0.2018, 0.158, 0.1537, 0.1307, 0.0935...","[0.2155, 0.2833, 0.17830000000000001, 0.1728, ...","[0.296, 0.3267, 0.16240000000000002, 0.1586000...","[0.1668, 0.2242, 0.13620000000000002, 0.1681, ...","[0.2119, 0.3388, 0.2033, 0.20500000000000002, ...","[0.2474, 0.8589, 0.8335, 0.7926000000000001, 0...","[0.5935, 1.4252, 0.6148, 0.5851000000000001, 0...","[1.9911, 2.2914, 0.6934, 0.5806, 0.51250000000...","[0.4022, 0.978, 0.49150000000000005, 0.8541000...","[0.3637, 2.0241, 0.9562, 0.9509000000000001, 0...","[5, 5, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 5, 5, 4, ..."
262,id_1aea9d2f3,GGAAAAGGUGCGAGGAGGUCAAAGACAGCCUGGCACCAAAAGGCAC...,......(((((.(((..(((...)))..))).)))))....(((((...,EEEEEESSSSSISSSIISSSHHHSSSIISSSISSSSSMMMMSSSSS...,2.072,1,107,68,"[0.6445000000000001, 0.7951, 0.3916, 0.4588000...","[0.7412000000000001, 1.0824, 0.4169, 0.6236, 0...","[0.8548, 1.1743999999999999, 0.377700000000000...","[0.6019, 0.8812000000000001, 0.344000000000000...","[0.579, 1.0959, 0.4544, 0.6791, 0.5169, 0.2052...","[1.3052000000000001, 3.2113, 0.52, 0.8717, 0.8...","[0.7443000000000001, 4.499, 0.3176, 1.3463, 1....","[2.1378, 7.0982, 0.2587, 2.5525, 1.1039, 0.046...","[1.1077, 4.7949, 0.4036, 2.2324, 0.77940000000...","[0.1859, 4.3782, 0.3559, 1.4065, 0.7923, -0.06...","[5, 5, 3, 3, 3, 3, 5, 5, 6, 5, 4, 5, 3, 5, 5, ..."
418,id_2bf4417d3,GGAAAUAGCACCACAUAAGACGGCAGAAGGUUUUUAUACACUCUAC...,..........((..((((..(.(((((..((......))..))).....,EEEEEEEEEESSIISSSSIISISSSSSIISSHHHHHHSSIISSSBB...,6.883,0,107,68,"[0.1109, 0.18660000000000002, 0.14750000000000...","[0.2479, 0.3582, 0.1257, 0.14980000000000002, ...","[0.203, 0.24910000000000002, 0.086600000000000...","[0.1221, 0.23850000000000002, 0.0743, 0.1278, ...","[0.1655, 0.2944, 0.1494, 0.1544, 0.1874, 0.154...","[0.2112, 1.4112, 1.0721, 0.7301000000000001, 1...","[0.7827000000000001, 2.7814, 0.224900000000000...","[1.4585, 2.8198, 0.2679, 0.4304, 0.99230000000...","[0.2795, 2.745, 0.1849, 0.7727, 1.4396, 1.0775...","[0.3003, 2.6887, 0.6883, 0.7201000000000001, 1...","[5, 5, 3, 3, 3, 6